In [1]:
from pathlib import Path
import shutil

# The path to the folder where the FoldX files are located
PATH_TO_FOLDX_FILES = Path().home() / "foldx"

# The path to where this notebook is located
THIS_DIR = Path().resolve()

# Creating a working directory for foldx's files
WORKING_DIR = THIS_DIR / "tmp"
WORKING_DIR.mkdir(exist_ok=True)

# Copying the rotabase.txt file to the working directory
shutil.copyfile(PATH_TO_FOLDX_FILES / "rotabase.txt", WORKING_DIR / "rotabase.txt")


PosixPath('/Users/sjt972/Projects/poli-docs/docs/poli-docs/understanding_foldx/01-single-mutation-using-foldx/tmp/rotabase.txt')

In [2]:
import urllib.request

# Downloading the PDB file
web_address = "https://files.rcsb.org/view/101M.pdb"
urllib.request.urlretrieve(
    web_address,
    WORKING_DIR / "101m.pdb"
)

(PosixPath('/Users/sjt972/Projects/poli-docs/docs/poli-docs/understanding_foldx/01-single-mutation-using-foldx/tmp/101m.pdb'),
 <http.client.HTTPMessage at 0x108184b20>)

In [3]:
# In case you choose a different pdb, you can change the name here
pdb_name = "101m"

In [4]:
import subprocess

# Building the command for repairing the wildtype PDB
if not (WORKING_DIR / "101m_Repair.pdb").exists():
    repair_cmd = [
        str(PATH_TO_FOLDX_FILES / "foldx"),
        "--pdb",
        "101m.pdb",
        "--command",
        "RepairPDB",
        "--water",
        "-CRYSTAL",
        "--pH",
        "7.0",
    ]

    # Running the repair command
    subprocess.run(repair_cmd, cwd=WORKING_DIR)

In [5]:
from Bio import PDB

parser = PDB.PDBParser()
structure = parser.get_structure("pdb", WORKING_DIR / "101m_Repair.pdb")
residues = list(structure.get_residues())

/Users/sjt972/anaconda3/envs/poli-docs-310/lib/python3.10/site-packages/Bio/PDB/Atom.py:241: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/sjt972/anaconda3/envs/poli-docs-310/lib/python3.10/site-packages/Bio/PDB/Atom.py:241: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/sjt972/anaconda3/envs/poli-docs-310/lib/python3.10/site-packages/Bio/PDB/Atom.py:241: PDBConstructionWarning: Used element 'C' for Atom (name=C) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/sjt972/anaconda3/envs/poli-docs-310/lib/python3.10/site-packages/Bio/PDB/Atom.py:241: PDBConstructionWarning: Used element 'O' for Atom (name=O) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/sjt972/anaconda3/envs/poli-docs-310/lib/python3.10/site-packages/Bio/PDB/Atom.py:241: PDBConstructionWarning: Used e

In [6]:
first_residue = residues[0]
print(f"Residue's name: {first_residue.resname}")
print(f"The sequence index: {first_residue.id[1]}")
print(f"Chain id: {first_residue.get_parent().id}")

Residue's name: MET
The sequence index: 0
Chain id: A


In [7]:
from Bio.SeqUtils import seq1

print(f"Residue as one letter: {seq1(first_residue.resname)}")

Residue as one letter: M


In [8]:
mutation_list = [
    {
        "residue_idx": 0,
        "to": "G",
    }
]

In [9]:
with open(WORKING_DIR / "individual_list.txt", "w") as f:
    for mutation in mutation_list:
        # We get the initial residue from the PDB file, this is a Residue type.
        # You can check the documentation of these inside Biopython.
        residue = residues[mutation["residue_idx"]]

        # The original residue lies in residue.resname. This is the 3-letter code.
        # To tranform it into the 1-letter code, we use seq1 from Bio.SeqUtils.
        original_residue = seq1(residue.resname)

        # We can read the position as the second position of the ID:
        # residue.id = (' ', position_in_chain, ' ')
        position = residue.id[1]

        # The chain ID can be read from the parent of the residue
        chain_id = residue.get_parent().id

        # The line we need to write, then, is:
        f.write(f"{original_residue}{chain_id}{position}{mutation['to']};" + "\n")

In [10]:
foldx_cmd = [
    str(PATH_TO_FOLDX_FILES / "foldx"),
    "--pdb",
    "101m_Repair.pdb",
    "--command",
    "BuildModel",
    "--mutant-file",
    str(WORKING_DIR / "individual_list.txt"),
    "--water",
    "-CRYSTAL",
    "--pH",
    "7.0",
]

In [11]:
subprocess.run(foldx_cmd, cwd=WORKING_DIR)

   ********************************************
   ***                                      ***
   ***             FoldX 5 (c)            ***
   ***                                      ***
   ***     code by the FoldX Consortium     ***
   ***                                      ***
   ***     Jesper Borg, Frederic Rousseau   ***
   ***    Joost Schymkowitz, Luis Serrano   ***
   ***    Peter Vanhee, Erik Verschueren    ***
   ***     Lies Baeten, Javier Delgado      ***
   ***       and Francois Stricher          ***
   *** and any other of the 9! permutations ***
   ***   based on an original concept by    ***
   ***   Raphael Guerois and Luis Serrano   ***
   ********************************************

1 models read: 101m_Repair.pdb
1 models read: 101m_Repair.pdb



BackHbond       =               -142.58
SideHbond       =               -48.61
Energy_VdW      =               -179.63
Electro         =               -8.33
Energy_SolvP    =               245.28
Energy_SolvH    =               -238.89
Energy_vdwclash =               3.42
energy_torsion  =               6.70
backbone_vdwclash=              158.16
Entropy_sidec   =               105.87
Entropy_mainc   =               231.69
water bonds     =               0.00
helix dipole    =               -8.75
loop_entropy    =               0.00
cis_bond        =               0.00
disulfide       =               0.00
kn electrostatic=               0.00
partial covalent interactions = 0.00
Energy_Ionisation =             1.56
Entropy Complex =               0.00
-----------------------------------------------------------
Total          = 				  -32.28




BackHbond       =               -142.58
SideHbond       =               -48.61
Energy_VdW      =               -179.63
Electro         =               -8.33
Energy_SolvP    =               245.28
Energy_SolvH    =               -238.89
Energy_vdwclash =               3.42
energy_torsion  =               6.70
backbone_vdwclash=              158.16
Entropy_sidec   =               105.87
Entropy_mainc   =               231.69
water bonds     =               0.00
helix dipole    =               -8.75
loop_entropy    =               0.00
cis_bond        =               0.00
disulfide       =               0.00
kn electrostatic=               0.00
partial covalent interactions = 0.00
Energy_Ionisation =             1.56
Entropy Complex =               0.00
-----------------------------------------------------------
Total          = 				  -32.28

Starting BuildModel
Reading MA0G;
Residue to Mutate META0 has residue index 0
Mutating residue = META0 into GLY


Your file run OK
End time of FoldX: Thu Nov 28 16:54:12 2024
Total time spend: 18.86 seconds.
validated file "101m_Repair_1.pdb" => successfully finished
Cleaning BuildModel...DONE


CompletedProcess(args=['/Users/sjt972/foldx/foldx', '--pdb', '101m_Repair.pdb', '--command', 'BuildModel', '--mutant-file', '/Users/sjt972/Projects/poli-docs/docs/poli-docs/understanding_foldx/01-single-mutation-using-foldx/tmp/individual_list.txt', '--water', '-CRYSTAL', '--pH', '7.0'], returncode=0)

In [12]:
# The details of these can be found in:
# https://foldxsuite.crg.eu/command/BuildModel

column_names = [
    "Pdb",
    "total energy",
    "Backbone Hbond",
    "Sidechain Hbond",
    "Van der Waals",
    "Electrostatics",
    "Solvation Polar",
    "Solvation Hydrophobic",
    "Van der Waals clashes",
    "entropy sidechain",
    "entropy mainchain",
    "sloop_entropy", 
    "mloop_entropy",
    "cis_bond",
    "torsional clash",
    "backbone clash",
    "helix dipole",
    "water bridge",
    "disulfide",
    "electrostatic kon",
    "partial covalent bonds",
    "energy Ionisation",
    "Entropy Complex"
]

In [13]:
import pandas as pd

with open(WORKING_DIR / f"Raw_{pdb_name}_Repair.fxout") as f:
    lines = f.readlines()

# The important data is in the last two lines
df = pd.DataFrame(
    [line.split() for line in lines[-2:]],
    columns=column_names
)

In [14]:
df[["Pdb", "total energy"]]

,Pdb,total energy
0,101m_Repair_1.pdb,-31.7457
1,WT_101m_Repair_1.pdb,-34.3436


In [15]:
from Bio.PDB import SASA

# Let's load up the structure again, for both the wildtype and the mutant
parser = PDB.PDBParser(QUIET=True)
wt_structure = parser.get_structure("pdb", WORKING_DIR / "101m_Repair.pdb")
mut_structure = parser.get_structure("pdb1", WORKING_DIR / "101m_Repair_1.pdb")

# Now, we can create a SASA computer
sasa = SASA.ShrakeRupley()

# We can attach, to each structure, its SASA
sasa.compute(wt_structure, level="S")
sasa.compute(mut_structure, level="S")

print(f"Wildtype SASA: {wt_structure.sasa}")
print(f"Mutant SASA: {mut_structure.sasa}")

Wildtype SASA: 8407.731560227876
Mutant SASA: 8439.063468009845
